# Using a single Table vs EArray + Table

The PyTables community keep asking what can be considered a FAQ.  Namely, should I use a single Table for storing my data, or should I split it in a Table and an Array?

Although there is not a totally general answer, the study below address this for the common case where one have 'raw data' and other data that can be considered 'meta'.  See for example: https://groups.google.com/forum/#!topic/pytables-users/vBEiaRzp3gI

In [1]:
import numpy as np
import tables

In [2]:
LEN_PMT = int(1.2e6)
NPMTS = 12
NEVENTS = 10

In [3]:
!rm PMT*.h5

In [4]:
def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

x = np.linspace(0, 1, 1e7)
rd = (gaussian(x, 1, 1.) * 1e6).astype(np.int32)

def raw_data(length):
    # Return the actual data that you think it represents PM waveforms better 
    #return np.arange(length, dtype=np.int32)
    return rd[:length]

## Using Tables to store everything

In [5]:
class PMTRD(tables.IsDescription):
    # event_id = tables.Int32Col(pos=1, indexed=True) 
    event_id = tables.Int32Col(pos=1)
    npmt = tables.Int8Col(pos=2)
    pmtrd = tables.Int32Col(shape=LEN_PMT, pos=3) 

In [6]:
def one_table(filename, filters):
    with tables.open_file("{}-{}-{}.h5".format(filename, filters.complib, filters.complevel), "w", filters=filters) as h5t:
        pmt = h5t.create_table(h5t.root, "pmt", PMTRD, expectedrows=NEVENTS*NPMTS)
        pmtr = pmt.row
        for i in range(NEVENTS):
            for j in range(NPMTS):
                pmtr['event_id'] = i
                pmtr['npmt'] = j
                pmtr['pmtrd'] = raw_data(LEN_PMT)
                pmtr.append()

In [7]:
# Using no compression
%time one_table("PMTs", tables.Filters(complib="zlib", complevel=0))

CPU times: user 428 ms, sys: 292 ms, total: 720 ms
Wall time: 717 ms


In [8]:
# Using Zlib (level 5) compression
%time one_table("PMTs", tables.Filters(complib="zlib", complevel=5))

CPU times: user 4.5 s, sys: 72 ms, total: 4.58 s
Wall time: 4.57 s


In [9]:
# Using Blosc (level 9) compression
%time one_table("PMTs", tables.Filters(complib="blosc", complevel=9))

CPU times: user 2.4 s, sys: 468 ms, total: 2.86 s
Wall time: 2.86 s


In [10]:
ls -sh *.h5

498M PMTs-blosc-9.h5  550M PMTs-None-0.h5   17M PMTs-zlib-5.h5


So, using no compression leads to best speed, whereas Zlib can compress data by ~32x.  Zlib is ~3x slower than using no compression though.  On its hand, the Blosc compressor is faster but it can barely compress the dataset.

## Using EArrays for storing raw data and Table for other metadata

In [11]:
def rawdata_earray(filename, filters):
    with tables.open_file("{}-{}.h5".format(filename, filters.complib), "w", filters=filters) as h5a:
        pmtrd = h5a.create_earray(h5a.root, "pmtrd", tables.Int32Atom(), shape=(0, NPMTS, LEN_PMT), expectedrows=NEVENTS)
        for i in range(NEVENTS):
            rdata = []
            for j in range(NPMTS):
                rdata.append(raw_data(LEN_PMT))
            pmtrd.append(np.array(rdata).reshape(1, NPMTS, LEN_PMT))
        pmtrd.flush()

In [12]:
# Using no compression
%time rawdata_earray("PMTAs", tables.Filters(complib="zlib", complevel=0))

CPU times: user 204 ms, sys: 328 ms, total: 532 ms
Wall time: 659 ms


In [13]:
# Using Zlib (level 5) compression
%time rawdata_earray("PMTAs", tables.Filters(complib="zlib", complevel=5))

CPU times: user 4.04 s, sys: 40 ms, total: 4.08 s
Wall time: 4.72 s


In [14]:
# Using Blosc (level 5) compression
%time rawdata_earray("PMTAs", tables.Filters(complib="blosc", complevel=9))

CPU times: user 344 ms, sys: 68 ms, total: 412 ms
Wall time: 412 ms


In [15]:
!ls -sh *.h5

 15M PMTAs-blosc.h5  6,0M PMTAs-zlib.h5    550M PMTs-None-0.h5
571M PMTAs-None.h5   498M PMTs-blosc-9.h5   17M PMTs-zlib-5.h5


We see that by using the Blosc compressor one can achieve around 10x faster output operation wrt Zlib, although the compression ratio can be somewhat smaller (but still pretty good).

In [16]:
# Add the event IDs in a separate table in the same file
class PMTRD(tables.IsDescription):
    # event_id = tables.Int32Col(pos=1, indexed=True) 
    event_id = tables.Int32Col(pos=1)
    npmt = tables.Int8Col(pos=2)

def add_table(filename, filters):
    with tables.open_file("{}-{}.h5".format(filename, filters.complib), "a", filters=filters) as h5a:
        pmt = h5a.create_table(h5a.root, "pmt", PMTRD)
        pmtr = pmt.row
        for i in range(NEVENTS):
            for j in range(NPMTS):
                pmtr['event_id'] = i
                pmtr['npmt'] = j
                pmtr.append()

In [17]:
# Using no compression
%time add_table("PMTAs", tables.Filters(complib="zlib", complevel=0))

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 4.3 ms


In [18]:
# Using Zlib (level 5) compression
%time add_table("PMTAs", tables.Filters(complib="zlib", complevel=5))

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 3.71 ms


In [19]:
# Using Blosc (level 9) compression
%time add_table("PMTAs", tables.Filters(complib="blosc", complevel=9))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 3.25 ms


In [20]:
!ls -sh *.h5

 15M PMTAs-blosc.h5  6,0M PMTAs-zlib.h5    550M PMTs-None-0.h5
571M PMTAs-None.h5   498M PMTs-blosc-9.h5   17M PMTs-zlib-5.h5


After adding the table we continue to see that a better compression ratio is achieved for EArray + Table with respect to a single Table.  Also, Blosc can make writing files significantly faster than not using compression (it has to write less).

## Retrieving data from a single Table

In [21]:
def read_single_table(complib, complevel):
    with tables.open_file("PMTs-{}-{}.h5".format(complib, complevel), "r") as h5t:
        pmt = h5t.root.pmt
        for i, row in enumerate(pmt):
            event_id, npmt, pmtrd = row["event_id"], row["npmt"], row["pmtrd"][:]
            if i % 20 == 0:
                print(event_id, npmt, pmtrd[0:5])

In [22]:
%time read_single_table("None", 0)

(0, 0, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
(1, 8, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
(3, 4, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
(5, 0, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
(6, 8, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
(8, 4, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
CPU times: user 76 ms, sys: 160 ms, total: 236 ms
Wall time: 235 ms


In [23]:
%time read_single_table("zlib", 5)

(0, 0, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
(1, 8, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
(3, 4, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
(5, 0, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
(6, 8, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
(8, 4, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
CPU times: user 748 ms, sys: 4 ms, total: 752 ms
Wall time: 750 ms


In [24]:
%time read_single_table("blosc", 9)

(0, 0, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
(1, 8, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
(3, 4, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
(5, 0, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
(6, 8, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
(8, 4, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
CPU times: user 936 ms, sys: 88 ms, total: 1.02 s
Wall time: 1.02 s


As Blosc could not compress the table, it has a performance that is worse (quite worse actually) to the uncompressed table.  On its hand, zlib can be more than 3x slower for reading than without compression.

## Retrieving data from the EArray + Table

In [25]:
def read_earray_table(complib, complevel):
    with tables.open_file("PMTAs-{}.h5".format(complib, "r")) as h5a:
        pmt = h5a.root.pmt
        pmtrd_ = h5a.root.pmtrd
        for i, row in enumerate(pmt):
            event_id, npmt = row["event_id"], row["npmt"]
            pmtrd = pmtrd_[event_id, npmt]
            if i % 20 == 0:
                print(event_id, npmt, pmtrd[0:5])

In [26]:
%time read_earray_table("None", 0)

(0, 0, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
(1, 8, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
(3, 4, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
(5, 0, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
(6, 8, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
(8, 4, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
CPU times: user 84 ms, sys: 116 ms, total: 200 ms
Wall time: 201 ms


In [27]:
%time read_earray_table("zlib", 5)

(0, 0, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
(1, 8, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
(3, 4, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
(5, 0, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
(6, 8, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
(8, 4, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
CPU times: user 1.35 s, sys: 12 ms, total: 1.36 s
Wall time: 1.36 s


In [28]:
%time read_earray_table("blosc", 9)

(0, 0, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
(1, 8, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
(3, 4, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
(5, 0, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
(6, 8, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
(8, 4, array([606530, 606530, 606530, 606530, 606530], dtype=int32))
CPU times: user 248 ms, sys: 4 ms, total: 252 ms
Wall time: 250 ms


## Conclusions

So, the EArray + Table takes a similar time to read than a pure Table approach.  However, the very improved compression capabilities of the EArray + Table makes this approach very appealing, most specially when using Blosc, which adds very small overhead when compared with not using compression at all.

The main conclusion here is that, whenever you have a lot of data to dump (typically in the form of an array), a combination of an EArray + Table is preferred instead of a single Table.  The reason for this is that HDF5 can store the former arrangement more efficiently, and that fast compressors like Blosc works way better too.

Finally, although meaningful, this experiment is based on a pure synthetic dataset.  It is always wise to use your own data in order to get your own conclusions.  It is specially recommended to have a look at the different compressors that comes with PyTables and see which one fits better to your needs: http://www.pytables.org/usersguide/libref/helper_classes.html